In [20]:
import numpy as np 
from tqdm import tqdm
import pandas as pd
import random
import ast


In [21]:
df = pd.read_csv('data/train.csv') # loading training data
data = []
for index, row in tqdm(df.iterrows()):
    temp = ast.literal_eval(row['tagged_sentence'])
    sent = []
    for word, tag in temp:
        if (len(sent)==0):
            sent.append((word.lower(), tag))
        else:
            sent.append((word, tag))
    data.append(list(reversed(sent))) # changing data-type of entries from 'str' to 'list'

47340it [00:04, 10097.81it/s]


In [22]:
# create train_test split
random.shuffle(data)
num_samples = len(data)
# train_data = data[:int(num_samples*0.8)]
# val_data = data[int(num_samples*0.8):]
train_data = data

In [23]:
df = pd.read_csv('data/test_small.csv') # loading test data
test_data = {}
for index, row in tqdm(df.iterrows()):
    test_data[row['id']] = ast.literal_eval(row['untagged_sentence']) # changing data-type of entries from 'str' to 'list'

4000it [00:00, 21770.27it/s]


In [24]:
# store words set, tags set as present in original corpus
words = set()
for sent in train_data:
    for word, _ in sent:
        words.add(word)
words =list(words)
num_words = len(words)
word2idx = {word: i for i, word in enumerate(words)}

# store word cnts
word_cnts = np.zeros(num_words)
for sent in train_data:
    for word, _ in sent:
        word_cnts[word2idx[word]]+=1
        
# define cutoff
cutoff = 2
unk = "UNK"
default ="NN"
eps = 0
# re define words
new_words = set()
for i in range(num_words):
    if (word_cnts[i]>cutoff):
        new_words.add(words[i])
    else:
        new_words.add(unk)
words = new_words
# define 
tags = set()
for sent in train_data:
    for word, tag in sent:
        # if word not in words:
        #     tags.add(unk)
        # else:
        #     tags.add(tag)
        tags.add(tag)

num_words = len(words)
words = list(words)
word2idx = {word: i for i, word in enumerate(words)}

tags.add(unk)
num_tags = len(tags)
tags =list(tags)
tag2idx = {tag: i for i, tag in enumerate(tags)}

assert(unk in words)
# assert(unk in tags)

In [25]:
# get unique words
words_tags_dict = dict()

for sent in train_data:
    for word, tag in sent:
        if word not in words_tags_dict:
            words_tags_dict[word] = set()
        words_tags_dict[word].add(tag)
        
unique_tag_words = []
for word in words_tags_dict:
    if (len(words_tags_dict[word]) == 1):
        unique_tag_words.append(word)


In [26]:
candidate_prefs = dict()
candidate_suffs = dict()

def get_prefs(word):
    n = len(word)
    # take upto 5 
    if (n<6):
        word = word+(6-n)*(" ")
    return [word[:1], word[:2], word[:3], word[:4], word[:5], word[:6]]

def get_suffs(word):
    n = len(word)
    # take upto 5 
    if (n<6):
        word = (6-n)*(" ")+word
    return [word[-1:], word[-2:], word[-3:], word[-4:], word[-5:], word[-6:]]

for word in unique_tag_words:
# for word in words:
    prefs = get_prefs(word)
    for pref in prefs:
        if pref not in candidate_prefs:
            candidate_prefs[pref] = dict()
        # for tag in list(words_tags_dict[word]):
        tag = list(words_tags_dict[word])[0]
        if tag not in candidate_prefs[pref]:
            candidate_prefs[pref][tag] = 0
        candidate_prefs[pref][tag]+=1
    
    suffs = get_suffs(word)
    for suff in suffs:
        if suff not in candidate_suffs:
            candidate_suffs[suff] = dict()
        tag = list(words_tags_dict[word])[0]
        if tag not in candidate_suffs[suff]:
            candidate_suffs[suff][tag] = 0
        candidate_suffs[suff][tag]+=1
        
pref_scores = dict()
suff_scores = dict()

for pref in candidate_prefs:
    mx = 0
    sm = 0
    mx_tag = None
    for tag, cnt in candidate_prefs[pref].items():
        if mx < cnt:
            mx_tag = tag
            mx = cnt
        sm += cnt
    score = mx/sm
    pref_scores[pref] = score, mx_tag

for suff in candidate_suffs:
    mx = 0
    sm = 0
    mx_tag = None
    for tag, cnt in candidate_suffs[suff].items():
        if mx < cnt:
            mx_tag = tag
            mx = cnt
        sm += cnt
    score = mx/sm
    suff_scores[suff] = score, mx_tag
    
def get_pred(word):
    best_pref=None
    best_pref_score = None
    pred_pref = None
    best_suff=None
    best_suff_score = None
    suffs = []
    prefs = get_prefs(word)
    suffs = get_suffs(word)
    
    for pref in prefs:
        if (pref not in pref_scores):
            continue
        if (best_pref == None):
            best_pref = pref
            best_pref_score = pref_scores[pref][0]
            pred_pref = pref_scores[pref][1]
            continue
        if (best_pref_score < pref_scores[pref][0]):
            best_pref = pref
            best_pref_score = pref_scores[pref][0]
            pred_pref = pref_scores[pref][1]
    
    for suff in suffs:
        if (suff not in suff_scores):
            continue
        if (best_suff == None):
            best_suff = suff
            best_suff_score = suff_scores[suff][0]
            pred_suff = suff_scores[suff][1]
            continue
        if (best_suff_score < suff_scores[suff][0]):
            best_suff = suff
            best_suff_score = suff_scores[suff][0]
            pred_suff = suff_scores[suff][1]
            
    if (best_pref_score == None):
        if (best_suff_score == None):
            pred = unk
            # would never happen since atleast one letter always occurs
        else:
            pred = pred_suff
    else:
        if (best_suff_score == None):
            pred = pred_pref
        else:
            if (best_pref_score < best_suff_score):
                pred = pred_suff
            else:
                pred = pred_pref
    
    return pred


In [27]:
## generate transition probabilities
transition_probs = np.zeros((num_tags, num_tags))
emission_probs = np.zeros((num_tags, num_words))
start_probs = np.zeros(num_tags)
for sent in train_data:
    last = None
    for word, tag in sent:
        if word not in word2idx:
            pred = get_pred(word)
            if (last):
                transition_probs[tag2idx[last], tag2idx[pred]]+=1
            else:
                start_probs[tag2idx[pred]]+=1
            last = pred
        else:
            if (last):
                transition_probs[tag2idx[last], tag2idx[tag]]+=1
            else:
                start_probs[tag2idx[tag]]+=1
            last = tag
        if word not in word2idx:
            emission_probs[tag2idx[pred], word2idx[unk]] += 1
        else:
            emission_probs[tag2idx[tag], word2idx[word]] += 1
            

# normalize
transition_probs = transition_probs/(transition_probs.sum(axis=1, keepdims=True)+1e-3)
emission_probs = emission_probs/(emission_probs.sum(axis=1, keepdims=True)+1e-3)
start_probs = start_probs/start_probs.sum()
        

In [28]:
# '[' in pref_scores

In [29]:
def custom_tagger_util(untagged_sentence):
    tagged_sentence = []
    T = len(untagged_sentence)
    viterbi_matrix = np.zeros((num_tags, T))
    backpointer_matrix = np.zeros((num_tags, T), dtype=int)

    # Initialization step
    if untagged_sentence[0] not in word2idx:
        viterbi_matrix[:, 0] = (start_probs+eps) *(emission_probs[:, word2idx[unk]]+eps)
    else:
        viterbi_matrix[:, 0] = (start_probs+eps) * (emission_probs[:, word2idx[untagged_sentence[0]]]+eps)

    # Recursion step
    for t in range(1, T):
        for s in range(num_tags):
            # Compute the maximum probability and corresponding backpointer
            if untagged_sentence[t] not in word2idx:
                probabilities = (viterbi_matrix[:, t - 1]+eps) * (transition_probs[:, s]+eps) * (emission_probs[s, word2idx[unk]]+eps)
            else:
                probabilities = (viterbi_matrix[:, t - 1]+eps) * (transition_probs[:, s]+eps) * (emission_probs[s, word2idx[untagged_sentence[t]]]+eps)
            backpointer_matrix[s, t] = np.argmax(probabilities)
            viterbi_matrix[s, t] = np.max(probabilities)

    # Termination step
    best_last_state = np.argmax(viterbi_matrix[:, T - 1])

    # Backtrace to find the most probable state sequence
    best_state_sequence = [best_last_state]
    for t in range(T - 1, 0, -1):
        best_last_state = backpointer_matrix[best_last_state, t]
        best_state_sequence.insert(0, best_last_state)
    
    tagged_sentence = []
    for i in range(T):
        # if tags[best_state_sequence[i]] == unk:
        #     tag = "NN"
        # else:
        #     tag = tags[best_state_sequence[i]]
        if untagged_sentence[i] not in word2idx:
            tag = get_pred(untagged_sentence[i])
        else:
            tag = tags[best_state_sequence[i]]
        if tag == unk:
            tag = default
        tagged_sentence.append((untagged_sentence[i], tag))
    return tagged_sentence
    

In [30]:
# ## validation ##
# corr = 0
# total = 0
# mistakes = []
# for sent in val_data:
#     untagged_sentence = []
#     for word, tag in sent:
#         if len(untagged_sentence) == 0:
#             untagged_sentence.append(word.lower())
#         else:
#             untagged_sentence.append(word)
#     pred = custom_tagger_util(untagged_sentence)
#     cnt = 0
#     for i in range(len(pred)):
#         if pred[i][1] ==':-':
#             cnt+=1
#         else:
#             cnt=0
#         if cnt==2:
#             # cut at this point
#             pred2 = custom_tagger_util(untagged_sentence[i:])
#             pred = pred[:i]+pred2
#             break
        
#     cnt = 0
#     temp = []
#     for i in range(len(sent)):
#         total+=1
#         assert(sent[i][0] == pred[i][0])
#         if (sent[i][1] == pred[i][1]):
#             corr+=1
#         else:
#             cnt+=1
#         temp.append((sent[i][0], sent[i][1], pred[i][1]))
#     mistakes.append((cnt, temp))
# mistakes = list(reversed(sorted(mistakes)))

In [31]:
# corr/total

In [32]:
# print(mistakes[3])

In [33]:
submission = {'id': [], 'tagged_sentence' : []} # dictionary to store tag predictions
# NOTE ---> ensure that tagged_sentence's corresponing 'id' is same as 'id' of corresponding 'untagged_sentence' in training data
def store_submission(sent_id, tagged_sentence):
    global submission
    if(sent_id in list(submission['id'])):
        return
    submission['id'].append(sent_id)
    submission['tagged_sentence'].append(tagged_sentence)
    
def clear_submission():
    global submission
    submission = {'id': [], 'tagged_sentence' : []}

In [34]:
for sent_id in tqdm(list(test_data.keys())):
    sent = list(reversed(test_data[sent_id]))
    sent[0] = sent[0].lower()
    tagged_sentence = custom_tagger_util(sent)
    cnt = 0
    for i in range(len(tagged_sentence)):
        if tagged_sentence[i][1] ==':-':
            cnt+=1
        else:
            cnt=0
        if cnt==2:
            # cut at this point
            tagged_sentence2 = custom_tagger_util(sent[i:])
            tagged_sentence = tagged_sentence[:i]+tagged_sentence2
            break
        
    store_submission(sent_id, list(reversed(tagged_sentence)))

100%|██████████| 4000/4000 [00:21<00:00, 187.39it/s]


In [35]:
import os
path = 'data/submission_rev.csv'
if (os.path.exists(path)):
    os.remove(path)
pd.DataFrame(submission).to_csv(path, index = False)